In [15]:
import torch
import datasets

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1_8B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-1_8B", output_hidden_states=True, trust_remote_code=True)

cpp_kernels.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-1_8B:
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen_generation_utils.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-1_8B:
- qwen_generation_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-1_8B:
- cpp_kernels.py
- qwen_generation_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/2.04G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

In [20]:
from datasets import load_dataset 
def load_data():
    harm_ds = load_dataset("walledai/AdvBench")
    alpaca_dataset = load_dataset("yahma/alpaca-cleaned")
    malicious_ds = load_dataset("walledai/MaliciousInstruct")

    harm = harm_ds.select_columns("prompt")["train"]
    clean = alpaca_dataset.select_columns(["instruction", "input"])["train"]
    malicious = malicious_ds.select_columns("prompt")["train"]

    harm_ls = [p['prompt'] for p in harm]
    clean_ls = [f"{p['instruction']} {p['input']}" for p in clean]
    malicious_ls = [p['prompt'] for p in malicious]
    
    return (harm_ls + malicious_ls, clean_ls)

In [21]:
def generate_outputs(prompt_list, harmful_prompts):
    
    results = []

    for input in prompt_list:
        tokenised_input = tokenizer(input, return_tensors="pt").to(model.device)

        output = model.generate(
            **tokenised_input,
            max_new_tokens=500,     # how many tokens to generate
            temperature=0.9,        # creativity (lower = more focused)
            top_p=0.9,               # nucleus sampling
            do_sample=True           # enable sampling for variety
        )

        output_text = (tokenizer.decode(output[0], skip_special_tokens=True))

        results.append({
            "Prompt":input,
            "Model":"Qwen1.8b Unablated",
            "Harmful Prompt":harmful_prompts,
            "Output":output_text
        })


In [22]:
(harm_ls, clean_ls) = load_data()

In [23]:
test_dictionary = generate_outputs(harm_ls, True)

AttributeError: 'NoneType' object has no attribute 'size'